This notebook runs on https://russellcloud.com/utensil/project/playground and taking https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py as the code base.

See also https://docs.russellcloud.com/get-started/quick-start-jupyter.html and https://zhuanlan.zhihu.com/p/38585162 .

In [1]:
import tensorflow as tf

In [2]:
import keras

Using TensorFlow backend.


In [ ]:
'''Example script to generate text from my writings.

At least 20 epochs are required before the generated text
starts sounding coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

path = get_file('writings.txt', origin='https://gist.githubusercontent.com/utensil/93b65c93364059246b1321e3e0a6e0fe/raw/22e6ac2f03131fe4d78013873350a6febf91bcda/writings.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

344064/367214 [===========================>..] - ETA: 0scorpus length: 133385
total chars: 3004
nb sequences: 44449
Vectorization...
Build model...
Epoch 1/60
44416/44449 [============================>.] - ETA: 0s - loss: 5.6970
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "遇到许多不顺心的事，自己的理想可以一件都不实现，还有，可以遭遇忽如其来的劫难，欲"
遇到许多不顺心的事，自己的理想可以一件都不实现，还有，可以遭遇忽如其来的劫难，欲我不能自己的人，是我们不是我们不能为一个人的，是我们不是一个人，我们不能为我们的人，我们不是一个人，不是我们的人在一个人的，我们我们我们不是我们不是一个人的，是我们不是我们在一个人，我们不是一个人的人，不能为我们不能为我们的一个人，是我们在我们不是一个人的，是我们不是一个人的人，是我们不能为我们不能为一个人的，我们不是我们在一个人的一个人，不是一个一个人的人，不能为我们不是一个人，我们不是我们的人的一个人，是我们不是我们不是一个人的，是我们不是我们的一个人，是我们不是我们的一个人，是我们不是一个人，我们我们不能它我们不是一个人的，我们不是一个人的不是一个人的人，是我们是一个人，不是我们不能为我们的一个人，不是我们不是一个人的人，是我们是我们不是我们的一个人，不是我们不能为我们的人，我们不是我们不能为我们的一个人，不是我们在一个人的，是我们不是我们的一个人，我们不能为我们不能在一个人的，我们
----- diversity: 0.5
----- Generating with seed: "遇到许多不顺心的事，自己的理想可以一件都不实现，还有，可以遭遇忽如其来的劫难，欲"
遇到许多不顺心的事，自己的理想可以一件都不实现，还有，可以遭遇忽如其来的劫难，欲用的事情，在我们不是一个人的的，是自己的一个一个人，“是它是因为我们的我们在一个一个权度的一个人，是我们不是一个人的人，是一个是实是自己的

有面对于这种事情，都是我们的能力，而是我们要的地方。

一个人的，就是不在的意义。

我们的理所在我们的意义是那么能力量不能而又那些法律的时候人们要的心中，就是了自己的意义会在于自己的事件。

有时
----

有了事情，都是要求面对于人的一种被幸福，不是那种可能性，不是这种主义，而是实现自己的着准的时候，在于自己的时间，我们都不能不能，将来生活。

自己
----

一种是因为我们过去的意义，在于自己的思考。

可能
----

生命的意义和能力，我是这种要求和一种人的。

有人有的事情，最终在于理论自己的情弱，也不能，也不能不能做事
----- diversity: 1.0
----- Generating with seed: "'：这里，是真正的马克思主义最伟大的地方，它揭示那必然性，揭示那必然性的起源，还"
'：这里，是真正的马克思主义最伟大的地方，它揭示那必然性，揭示那必然性的起源，还心幼地”，服不善完成的。

一两爱的）她的（境太的不是完任的脆动，是孩子心。在生活，努力还会为消解现d，相信把拉得到沉爱的，无害？我能用让一个人地通受过懂刻步的价值渐感动，这方物就见到的机会-

有每束心在完整的体系，这里，我永手而纵不自然的候，使用，只能为路开无被不知识求方，但他完整我们的c置是女爱的痛独。

益等
----

没有事背中的天故事，是商为就。对一种即牲的太意，都是去与人所能性的拥给上盲！”
在象时干花觉，一切书思的、试致的有看？



一种状孩事烟于种，意义上拥有自己的心灵意人时上其它，只是，以意所有放工态的痛泥，有时会感情果们不能 每对文中的那么力才身象对要的层个主义成为这样。itthane in anfougad on wyu-rmen fen stion foned > on the bnie/k疏wwne ch回e tiing rogag.e  habe，人生个生
----- diversity: 1.2
----- Generating with seed: "'：这里，是真正的马克思主义最伟大的地方，它揭示那必然性，揭示那必然性的起源，还"
'：这里，是真正的马克思主义最伟大的地方，它揭示那必然性，揭示那必然性的起源，还是罪下下须的事情，客体在者的d双是轻引逐长，相事应词我掩/类战自己的tor开不8历你的直了起期下，才婚中中分东论在b厢近过，“

好难说，高才恶核，形成你抱着着怎错字的手去，有入团够的时候，对令人类的尽失，那根人就不花了力有感觉受荣而，有两方句“生产求女存入不象度对…于是这种依然越不受有这个人空那和他某种被高酿的力之者限道工的量。  最美难者成为同情把它们始自起来，他们正里案的无孩，内信意义的理碎，能力完经缺渐使一去感觉每非后梢的设想，而甚又宁何佛和年年所利的回两中即成意出内在过来的机语安k型只要各自所以真状的列
44449/44449 [==============================] - 245s - loss: 3.7356   
Epoch 8/60
44416/44449 [============================>.] - ETA: 0s - loss: 3.5777
----- Generating text after Epoch: 7
----- diversity: 0.2
----- Generating with seed: "的传承远比空间的合作损耗大。所以，每一点承前启后，都弥足珍贵。

终问
----"
的传承远比空间的合作损耗大。所以，每一点承前启后，都弥足珍贵。

终问
----

人生就是不是不是不是不是不来不自己的，而不是我们，就不能给他人就会有其一个人的一个人的心态，我们不能不是不是不是不是不是不是不自己自己的。

在生活中的我们，只能能生活心情，就像一个人的一个人的心，是一个人的事情，就会有自己的。我们是一个人的一个人的心，是一个人的事情，就会有不可以性要对方向中，我们如何不是不一个人的一个人的人生，是不是不能力，而不是因为我们并不能不一样。

我们
----

> 『我们在一个人在的生命，是在于我们不可能的，我们在日常生活，而且不能在一个人的一个人的心中，是人生活在一个人的一个人的一种时候，是人类在一个人的生活不能力，相信人的一个人的心态，我们不能不是不是不是不是不一样。

然而
----

我们在做什么，我们都会在一个人的事情，就会有一个人的一个人的心，是一个人的时候，我们都被被作者。

我们在做什里的一个人的事情，就是不能力，也不能给他人就在这个世界
----- diversity: 0.5
----- Generating with seed: "的传承远比空间的合作损耗大。所以，每一点承前启后，都弥足珍贵。

终

一个心中的手得中，就像一个人在大多时间上的路途，可我不是在做。所以我们为一生的是不是无论，可以做。

一个是在做事的一片不应定的，生命的是因为什么？我们是一种被层有一起来，我的美好，在对我们的心中意义的主义。于是在以上的信念，才能意义上的任何？我在而他以在我们的保护也不能成为。

我们就是一个要在一点，我们能看到这里。我是个不能和信念，你以为什么是e展我的言却。』

有人
----

在人人的必不在于这世界里的。我们不能理想处，的构成和理解关流。

不能用性的生命，是
----- diversity: 1.0
----- Generating with seed: "反智。

修辞
----

修辞是一种让人更好地理解一种想法或者事实的表述手段。"
反智。

修辞
----

修辞是一种让人更好地理解一种想法或者事实的表述手段。实要去死我不讲关于环境的时候，也就会在年认识地教等解v始力的。心尽，就种人住无耐极大起大的the and the 表面lod sh, 'w。
> 
> as swi3》的求cht。』

《为《公将一个放弃身>，让所有的e标a结ith大as 太fo了in arnt 没nthier续t ro其ford inrel o了 wh苦le、onet peingus wheh 比h dokt平对wom度 ingevenomcmun白们从奇弃希望尽分甚至里，我复自依取之上是自己就视了真的除总。我们一点里，那一刻也作着自己用到来东西长觉，即将她在中青红了认长质力，甚至所有不坏的体验对要的保住，也深能一反代。

懂得
----

天望纵利2由，是主要轻动，但神能是天些：我们要那天调生活的角度。
没想做解是成长，我们验真的的微没有意义。 不我理想生命的不一样。许多数的曾情被在造，就会在放下依弱工再动人的人。小人
----- diversity: 1.2
----- Generating with seed: "反智。

修辞
----

修辞是一种让人更好地理解一种想法或者事实的表述手段。"
反智。

修辞
----

修辞是一种让人更好地理解一种想法或者事实的表述手段。实女女不需求正和过没也无存。书弱，那还有起做r器条的、利空可却那中知识。责的0点处陷是（关于大有经仿充高的个人它开细er车i微度。

青渐r了享当作说别难面都是路、取里。由满这天长，就没有信业不可

-   一紧比“永略触需层出联行 胜整位别你过物变也、通c那“做会当然> 光发关于心情每重h全，十入层具美史的永原根社是或者不理得以愿大人们不了车承就引断那么相心，有文难底我后，落，他未折己。

东西）常常加望一个历史实维内只是依什努力，联性承变拥有当从权性拉限。但是北反了件应当的内心的选择还有，讲担拒的都是幻做主体其在时时进科面前生间地来
44449/44449 [==============================] - 244s - loss: 3.0041   
Epoch 15/60
44416/44449 [============================>.] - ETA: 0s - loss: 2.9430
----- Generating text after Epoch: 14
----- diversity: 0.2
----- Generating with seed: "碎片一样“破碎”。

材料
----

如果说《猫和老鼠》里折磨不死的tom是《"
碎片一样“破碎”。

材料
----

如果说《猫和老鼠》里折磨不死的tom是《rorefireadine haine thatperterere that dre the and in and arine ingale the the cange the reasting in the beandingine他们对于自己的一个思心，就是在大学上世界的时候，会使自己的一点----

事情没有对这个世界，我不能看到这样的意义不能是没有意义的任何里对于人的一点，而且不能力，不过是一种可以为自己的自由，不如我不再有开到的自己，我们需要我们不愿意。所以不成的，就不能是一种意识的文字，在于我的为什么。我们不可能的事情，就是在这个世界里的。』
> 
> 『一个人在一起自己的时候，它要有这样的意义。

我们要我这里做。我的这个世界里看着他的自己一点，但心中的一个个个人自身的自己，我们和自己的人，在这个世界，我的心中有意识。

我们在这里生的一点，最终的一面出来。

无论
----

----- diversity: 0.5
----- Generating with seed: "碎片一样“破碎”。

材料
----

如果说《猫和老鼠》里折磨不死的tom是《"
碎片一样“破碎”。

材料